# https://iprogramming.bacpop.org/print.html
## counting codons

In [4]:
def countCodons(seqs):
    import numpy as np
    bases= ['A', 'C', 'G', 'T']
    codon_map= dict()
    idx= 0
    for base1 in bases:
        for base2 in bases: 
            for base3 in bases:
                codon= "".join((base1+base2+base3))
                codon_map[codon]= idx
                idx+= 1

    codons_len= int(len(seqs[0]) // 3)
    counts= np.zeros((codons_len, len(codon_map)), np.int32)
    for seq in seqs:
        for codon_idx in range(codons_len):
            codon= seq[codon_idx:(codon_idx+3)]
            if not "-" in codon:
                counts[codon_idx, codon_map[codon]]+= 1
    
    count_vec= list()
    for pos in counts:
        pos_vec= list()
        for codon in list(codon_map.keys()):
            if codon in pos:
                pos_vec.append(pos[codon])
            else:
                pos_vec.append(0)
            count_vec.append(pos_vec)
    
    return count_vec

In [5]:
import time
import gzip

def read_fasta_sample(fp):
    name, seq= None, []
    for line in fp:
        line= line.strip()
        if line.startswith(">"):
            if name: yield (name, "".join(seq).upper())
            name, seq= line[1:], []
        else:
            seq.append(line)
    if name: yield (name, "".join(seq).upper())

def read_file(file_name):
    n_samples= 0
    with open(file_name, "rb") as test_f:
        zipped= test_f.read(2) == b"\x1f\x8b" # gzip文件前两个字节标志
    if zipped:
        fh= gzip.open(file_name, "rt")
    else:
        fh= open(file_name, "rt")
    with fh as fasta:
        seqs= list()
        names= list()
        for h, s in read_fasta_sample(fasta):
            if len(s) % 3 != 0:
                raise RuntimeError(f"Sequence {h} is not a multiple of three")
            elif len(seqs)>0 and len(s) != len(seqs[0]):
                raise RuntimeError(f"Sequence {h} is length {len(s)}, expecting {len(seqs[0])}")
            else:
                seqs.append(s)
                names.append(h)
    return seqs, names

def main():
    start_t= time.time_ns()
    seqs, names= read_file("/share/home/lsy_luzhen/software/Git/data/self-training/Python/materials/BIGSdb_024538_1190028856_31182.dna.aln.gz")
    end_t= time.time_ns()
    time_ms= (end_t-start_t)/1000000
    print(f"time to read {len(seqs)} samples: {time_ms} ms")
    count_vec= countCodons(seqs)

if __name__== "__main__":
    main()


time to read 4889 samples: 81.163348 ms


## debugging
python has `pdb` built-in, we can run it as follows:
`python -m pdb software/Git/data/self-training/Python/pgm/intermediateResearchProgramming/002_example1.py`
(gdb abd lldb are equivalent debuggers for compiled code).
run `help` to see the command. Most useful are setting breakpoints with `b`; continuing execution with `c`, `r`, `s` and `n`; printing variabls with `p`.

by stopping the code on the line with `codon= "".join((base1+base2+base3))` by typing `b 56` to add a breakpoint, then `r` to run the code, we can have a look at some counts: `p pos`. We'll step through some of the next lines with `n`.

In [ ]:
codons_new = ["ATG", "GGG", "TTT"]
codons_correct = ["ATG", "GGG", "TTT"]

assert codons_new == codons_correct  # 条件为真，程序继续运行
print("密码子匹配正确！")

codons_new = ["ATG", "GGG", "CCC"]
assert codons_new == codons_correct  # 条件为假，抛出 AssertionError

## profiling
The top-down approach to profiling and optimization is to find the most time-consuming part of codes and to treat that as a target.

The bottom-up approach finds "hot" functions, which are the lower level parts of the code and frequently called by many other functions.

Using a profiler can help us get a much better sense of what to focus on when we want to make our code faster.

`python -m cProfile -o software/Git/data/self-training/Python/materials/cprof_results software/Git/data/self-training/Python/pgm/intermediateResearchProgramming/002_example1.py`

In [12]:
import pstats
p= pstats.Stats("/share/home/lsy_luzhen/software/Git/data/self-training/Python/materials/cprof_results")
p.sort_stats('cumulative').print_stats()

Mon Feb 24 15:27:49 2025    /share/home/lsy_luzhen/software/Git/data/self-training/Python/materials/cprof_results

         614349 function calls (614322 primitive calls) in 0.581 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      4/1    0.000    0.000    0.581    0.581 {built-in method builtins.exec}
        1    0.000    0.000    0.581    0.581 software/Git/data/self-training/Python/pgm/intermediateResearchProgramming/002_example1.py:1(<module>)
        1    0.000    0.000    0.567    0.567 software/Git/data/self-training/Python/pgm/intermediateResearchProgramming/002_example1.py:64(main)
        1    0.342    0.342    0.352    0.352 software/Git/data/self-training/Python/pgm/intermediateResearchProgramming/002_example1.py:36(countCodons)
        1    0.006    0.006    0.215    0.215 software/Git/data/self-training/Python/pgm/intermediateResearchProgramming/002_example1.py:15(read_file)
     4890    0.092    0.000   

## optimization strategies
### use a betteer algorithm or data structure
### avoiding recomputing results
### using arrays and preallocation

In [1]:
import numpy as np
print(np.array([1,2,3])+np.array([4,5,6]))
print([1,2,3]+[4,5,6])

[5 7 9]
[1, 2, 3, 4, 5, 6]


In [3]:
def setup():
    xlen= 1000
    ylen= 1000
    rand_mat= np.random.rand(xlen, ylen)
    lol= list()
    for x in range(xlen):
        sublist= list()
        for y in range(ylen):
            sublist.append(rand_mat[x,y])
        lol.append(sublist)
    return rand_mat, lol

def sum_mat(mat):
    return np.sum(mat)

def sum_lol(lol):
    from math import fsum
    row_sum= [fsum(x) for x in lol]
    full_sum= fsum(row_sum)
    return full_sum

def main():
    import timeit
    rand_mat, lol= setup()
    print(sum_mat(rand_mat))
    print(sum_lol(lol))

    setup_str= '''
import numpy as np
from __main__ import setup
from __main__ import sum_mat
from __main__ import sum_lol

rand_mat, lol= setup()
'''

    print(timeit.timeit("sum_mat(rand_mat)", setup=setup_str, number=100))
    print(timeit.timeit("sum_lol(lol)", setup=setup_str, number=100))

if __name__== "__main__":
    main()

499779.7098816839
499779.7098816838
0.03354182280600071
1.7209598952904344
